In [2]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

import librosa
import numpy as np
import os

In [3]:
parent_folder = "D:\MaiTexa\projects\Audio deepfake detection\Audio data"

In [4]:

# Function to extract features and labels from audio files in a folder
def extract_features_and_labels_from_folder(parent_folder):
    all_features = []
    all_labels = []

    for subfolder in os.listdir(parent_folder):
        subfolder_path = os.path.join(parent_folder, subfolder)
        if os.path.isdir(subfolder_path):
            features = extract_features_from_folder(subfolder_path)
            labels = [subfolder] * len(features)  # Assign label based on subfolder name
            all_features.extend(features)
            all_labels.extend(labels)

    return all_features, all_labels

In [5]:
# def extract_features_and_labels_from_folder(parent_folder, max_samples_per_class=1500):
#     all_features = []
#     all_labels = []
#     class_counts = {}

#     for subfolder in os.listdir(parent_folder):
#         subfolder_path = os.path.join(parent_folder, subfolder)
#         if os.path.isdir(subfolder_path):
#             if subfolder not in class_counts:
#                 class_counts[subfolder] = 0

#             for file_name in os.listdir(subfolder_path):
#                 if class_counts[subfolder] >= max_samples_per_class:
#                     break

#                 file_path = os.path.join(subfolder_path, file_name)
#                 try:
#                     y, sr = librosa.load(file_path)
#                     mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
#                     all_features.append(mfccs)
#                     all_labels.append(subfolder)  # Assign label based on subfolder name
#                     class_counts[subfolder] += 1
#                 except Exception as e:
#                     print("Error encountered while processing file:", file_path)

#     return all_features, all_labels


In [6]:
def extract_features_from_folder(folder_path):
    features = []
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        try:
            y, sr = librosa.load(file_path)
            mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
            features.append(mfccs)
        except Exception as e:
            print("Error encountered while processing file:", file_path)

    return features

In [7]:
all_features, all_labels = extract_features_and_labels_from_folder(parent_folder)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(all_features, all_labels, test_size=0.2, random_state=42)

In [9]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [10]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

Test Accuracy: 99.86%


In [11]:
model_filename = "random_forest_model5.joblib"
joblib.dump(model, model_filename)
print(f"Model saved as {model_filename}")

Model saved as random_forest_model5.joblib


In [12]:
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path)
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
        return np.array([mfccs])
    except Exception as e:
        print("Error encountered while processing file:", file_path)
        return None

In [13]:
# def runtest(example_file_path):
#     # Load the model
#     loaded_model = joblib.load("random_forest_model4.joblib")

#     # Extract features from the example file
#     example_features = extract_features(example_file_path)
#     if example_features is not None:
#         prediction = loaded_model.predict(example_features)
#         class_label = "Real" if prediction[0] == 1 else "Fake"
#         return f"{class_label} Audio File"
#     else:
#         return "Error extracting features from the example file."

In [14]:
# def runtest(example_file_path):
#     # Load the model
#     loaded_model = joblib.load("random_forest_model4.joblib")

#     # Extract features from the example file
#     example_features = extract_features(example_file_path)
#     if example_features is not None:
#         # Get the prediction
#         prediction = loaded_model.predict(example_features)
#         class_label = "Real" if prediction[0] == 1 else "Fake"
        
#         # Get the probability of the prediction
#         proba = loaded_model.predict_proba(example_features)
#         proba_fake, proba_real = proba[0]
        
#         return f"{class_label} Audio File with probability: Real {proba_real*100:.2f}%, Fake {proba_fake*100:.2f}%"
#     else:
#         return "Error extracting features from the example file."


In [15]:
def runtest(example_file_path):
    # Load the model
    loaded_model = joblib.load("random_forest_model5.joblib")

    # Extract features from the example file
    example_features = extract_features(example_file_path)
    if example_features is not None:
        # Get the probability of the prediction
        proba = loaded_model.predict_proba(example_features)
        proba_fake, proba_real = proba[0]
        
        # Determine the class with the maximum probability
        max_proba_class = "Real" if proba_real > proba_fake else "Fake"
        
        return f"{max_proba_class} Audio File with probability: Real {proba_real*100:.2f}%, Fake {proba_fake*100:.2f}%"
    else:
        return "Error extracting features from the example file."


In [17]:
print(runtest(r"C:\Maitexa1\projects\AI deepfake detection3\AudiodataMixed\fake\LJ001-0024_gen.wav"))


Fake Audio File with probability: Real 1.00%, Fake 99.00%
